 <font size="6">Обучение на реальных данных</font>

# Проблемы при работе с реальной задачей машинного обучения

В реальных задачах, особенно если вы работаете над новой проблемой, вы столкнетесь с широким спектром проблем. Приведем часть из них и сопроводим примером на основе задач нахождения клеток крови на фотографии мазка крови

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/bloods_cell.jpg" width="500px">

- **нехватка данных** - фотографий мазков крови может быть недостаточно для построения сложной модели с нуля

- **недостаток размеченных данных** - возможно, существует достаточно большое количество фотографий мазков крови (например, в историях болезни), но очень малая часть из них размечена

- **некачественная разметка** - мазок крови могли доверить анализировать студенту-практиканту. Размечать его мог вообще человек не из профессии - например, хотевший таким образом увеличить обучающую выборку для модели на конкурс kaggle. Даже в широко известных MNIST, CIFAR и ImageNet есть ошибки в разметке ([примеры](https://labelerrors.com/))


<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/wrong_imagenet_prediction.png" width="500px">


- **низкое качество данных** - не все фотографии будут в хорошем разрешении. Да и не все образцы были правильно подготовлены. А еще могут попадаться фотографии от пациентов, больных чем-нибудь экзотическим, в результате чего их клетки выглядят сильно отличительно от обычных. Или же в крови пациента просто плавают паразиты

Серповидная клеточная анемия приводит к аномальным эритроцитам

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/normal_and_sickle_shaped_erythrocytes.JPG" width="500px">


[источник](https://commons.wikimedia.org/wiki/File:Sicklecell3.jpg)




А так выглядит мазок крови при сонной болезни 

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/trypanosoma_among_red_blood_cells.jpg" width="500px">

  

- **несбалансированность датасета** - клетки крови встречаются в разных пропорциях. Какие классы могут быть плохо представлены (**минорные классы**) Потому в вашем датасете может быть всего 10 фотографий, на которых присутствуют базофилы. Нейросети будет очень заманчиво вообще не пытаться найти базофилы (всего 10 ошибок). 

<img src="https://edunet.kea.su/repo/EduNet-content/L11/out/leukocyte_blood_formula_table.png" >

- **полные дупликаты** - в данных могут быть полные дубликаты. Кто-то до вас агрегировал фотографии из разных источников и вы либо не обратили на это внимание, либо он забыл об этом сказать. Такие данные надо сразу помечать и использовать только после предварительного размышления, т.к. они могут мешать вам и на этапе обучения модели, и на итоговой валидации ее качества (если один и тот же объект попадет и в обучение, и в валидацию). 

- **неполные дубликаты** - в данных могут быть данные от одного и того же пациента. Кажется, что если это разные мазки крови - то ок. На самом деле и это уменьшает количество информации, которую может извлечь нейросеть из данных. С такими данными также нужно аккуратно работать и не допускать попадания одного пациента и в обучение, и в тест.

- **малое число источников данных** - проблема, родственная предыдущей. В вашем датасете могут быть данные только от одного микроскопа или одной модели микроскопа. Могут быть данные, снятые только одним специалистом или в одной больнице или только у взрослых (фотографий мазков детей нет). Это так же может влиять на способность вашего алгоритма генерализовывать полученное решение и требует пристального внимания. 


Все это приводит к целому спектру проблем, из которых самой типичной будет переобучение модели - какую простую б модель вы не взяли, она все равно будет выучивать искажения вашего датасета. 





# Общие подходы при работе с реальными данными


## Недостаток данных 

Если у вас мало данных - попробуйте найти еще данных для вашей задачи. 
Совет приводится во многих инструкциях по борьбе с малым количеством данных и может быть воспринят с юмором. 
Однако часто для вашей задачи действительно существуют данные, собранные другими людьми. Также часто можно найти данные, которые очень похожи на ваши и их можно использовать в обучении, но, например, учитывать с меньшим весом.
Даже 20 дополнительных примеров могут сильно облегчить ситуацию. 

Вы также можете использовать данные, которые не совсем похожи на ваши, в качестве внешней валидации. Тем самым вы можете разбивать свой изначальный датасет только для кросс-валидации, и не выделять отдельную часть для теста. Тестом послужат как раз "не совсем" похожие данные. 

Так же можно использовать две техники которые мы сегодня рассмотрим подробнее: Аугментация и Transfer Learning

# Баланс классов

Прежде всего надо убедиться что датасет сбалансирован ли наш датасет.

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine


def show_class_balance(y,classes):
  _, counts = torch.unique(torch.tensor(y),return_counts = True)
  plt.bar(classes,counts)
  plt.ylabel('n_samples')
  plt.show()

wine = load_wine()
X = wine.data 
y = wine.target
classes = wine.target_names

show_class_balance(y,classes)

На практике разница в 10-20% несущественна, однако мы воспользуемся этим датасетом что бы продемонстрировать техники борьбы с несбалансированными данными.

## Изменение баланса класса сэмплированием 

Если в данных недостаток именно конкретного класса, то можно бороться с этим при помощи разных способов сэмплирования. 

Важно понимать, что в большинстве случаев данные, полученные таким способом должны использоваться в качестве **обучающего набора**, но ни в коем случае не в качестве **валидации** или **теста**. 


### Дублирование примеров меньшего класса (oversampling)

Мы можем увеличить число объектов меньшего класса за счет дублирования. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/oversampling_scheme.png" width="600">

В этом случае наша модель будет "вынуждена" обращать внимание на минорный класс. 

Такой `Resampling` может быть выполнен с помощью пакета [imbalanced-learn](https://pypi.org/project/imbalanced-learn/), как показано ниже:



In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_ros, y_ros = ros.fit_resample(X, y)

show_class_balance(y_ros,classes)

### Уменьшение числа примеров бОльшего класса (undersampling)

Аналогично, можно взять для обучения не всех представителей бОльшего класса. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/undersampling_scheme.png" width="600"> 

Это также вынуждает модель обращать внимание на оба класса. 
Минус подхода очевиден - мы можем выбросить важных представителей бОльшего класса, ответственных за существенное улучшение генерализации,  и за счет этого качество модели существенно ухудшится. 
Можно пытаться выбрасывать объекты бОльшего класса как-то по-умному - например, кластеризовать объекты бОльшего класса и брать по заданному количеству объектов из каждого класса. 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)

show_class_balance(y_res,classes)

### Ансамбли + undersampling

Можно использовать ансамбли вместе с undersampling. В этом случае мы можем, к примеру, делать сэмплирование только бОльшего класса, а объекты минорного класса оставлять как есть. 


<img src="https://edunet.kea.su/repo/EduNet-content/L11/out/ensembles_and_undersampling.png" width="700px">

Или просто сэмплировать объектов и того, и другого класса равное количество.

### Балансирование представленности объектов в батчах

В случае нейросетей можно балансировать встречаемость каждого класса не на уровне датасета, а на уровне батча. Например, собираем каждый батч таким образом, чтобы в нем было поровну всех классов.

 Это может улучшать сходимость даже в случае небольшого дисбаланса или его отсутствия, т.к. мы будем избегать шагов обучения нейросети, в которых она просто не увидела какого-то класса в силу случайных причин.

<img src="https://edunet.kea.su/repo/EduNet-content/L11/out/batch_balancing.png" >


В Pytorch эту фнкциональность можно получить используя класс WeightedRandomSampler. Для его инициализации требуется рассчитать вес каждого класса. 

In [ ]:
import torch
# https://pytorch.org/docs/stable/generated/torch.unique.html

_, counts = torch.unique(torch.tensor(y),return_counts = True)
weights = counts.max() / counts
print('Классы: ', classes)
print('Веса классов: ', weights)

In [ ]:
from torch.utils.data import DataLoader,  TensorDataset, WeightedRandomSampler

tensor_x = torch.Tensor(X) # transform to torch tensor
tensor_y = torch.Tensor(y)
dataset = TensorDataset(tensor_x,tensor_y)

batch_size= 8

# Every sample must have a weight
weight_for_every_sample = []
for l in wine.target:
  weight_for_every_sample.append(weights[l].item())

sampler = WeightedRandomSampler(torch.tensor(weight_for_every_sample), len(dataset))
loader = DataLoader(dataset, batch_size= batch_size, shuffle = False, num_workers=1, sampler = sampler)

In [ ]:
print(len(dataset))
for data, labels in loader:
  print(labels, torch.unique(labels,return_counts = True)[1])

Confusion matrix

In [ ]:
# targets and preds must be calculated during training

from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( wine.data, wine.target, test_size=0.4, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
preds = model.predict(X_test)

conf_matrix = pd.DataFrame(confusion_matrix(y_test, preds))
print(wine.target_names)
conf_matrix.columns = wine.target_names#wine.classes
conf_matrix.index = wine.target_names

conf_matrix = conf_matrix.rename_axis('Real')
conf_matrix = conf_matrix.rename_axis('Predicted', axis='columns')

conf_matrix

### Генерация синтетических данных

Другой подход к решению этой проблемы - создание синтетических данных. Делать это можно по-разному. 



#### Генерация данных на основе имеющихся 





##### SMOTE

**Synthetic Minority Over-sampling Technique (SMOTE)** позволяет генерировать синтетические данные за счет реальных объектов из минорного класса. 

Алгоритм работает следующим образом:

1. Для случайной точки из минорного класса выбираем $k$ ближайших соседей из того же класса. 
2. Для первого соседа проводим отрезок, соединяющий его и выбранную точку. На этом отрезке случайно выбираем точку. 
3. Эта точка - новый **синтетический** объект минорного класса.
4. Повторяем процедуру для оставшихся соседей.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/generate_synthetic_data.png" width="700">

Число соседей, как и число раз, которое мы запускаем описанную выше процедуру можно регулировать.


### Изменение функции потерь
Модели в машинном обучении “ленивы”. При работе с несбалансированными классами модель будет чаще сталкиваться с доминирующим классом и вместо того, чтобы разбираться в признаках объектов, может начать ориентировать на статистическое распределение классов. 

Пример: датасет, в котором 95% объектов относятся к классу 1 и 5% к классу 0. Модель может выучиться “всегда говорить ~~да~~ класс 1” и в 95% случаях она будет права.

Чтобы это поправить, можно изменить функцию потерь, чтобы больше штрафовать модель за ошибки в минорных классах. Для этого можно:
- Добавлять веса в функцию потерь для компенсации дисбаланса классов.
Во многих функциях ошибки в PyTorch (например, [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)) есть параметр `weight`, который имеет по умолчанию значение `None`. В него можно передать тензор весов, соответствующий размеру вектора целевых значений и получить взвешенную функцию ошибок. Например, если у нас есть два класса с соотношением 4:1 можно выбрать `weight` = [0.2, 0.8].
- Можно менять форму функции ошибок. В 2017 году для работы с несбалансированными классами был предложен [Focal Loss](https://arxiv.org/abs/1708.02002?context=cs). Это - модификация cross-entropy loss, которая модифицирует ее [форму](https://medium.com/visionwizard/understanding-focal-loss-a-quick-read-b914422913e7) для различных классов. С тех пор появились различные [модификации](https://paperswithcode.com/method/focal-loss) функции ошибок с которыми можно и нужно эксперементировать.  



### Обнаружение аномалий / изменений

В случае сильно несбалансированных наборов данных, таких как fraud (мошенничество) или машинный сбой, стоит задуматься, могут ли такие примеры рассматриваться как аномалия (выброс) или нет. Если такое событие и впрямь может считаться аномальным, мы можем использовать такие модели, как `OneClassSVM`, методы кластеризации или методы обнаружения гауссовских аномалий.

Эти методы требуют изменения способа мышления: мы будем рассматривать аномалии, как отдельный класс выбросов. Это может помочь нам найти новые способы разделения и классификации.

Проблемой при работе с аномалиями является то, что аномальных значений может быть очень мало или вообще не быть. В таком случае алгоритм учит паттерны нормального поведения и реагирует на отличия от паттернов.

Разберем пример обнаружения аномалий с помощью `OneClassSVM` из библиотеки Sk-Learn (там же, можно найти еще множество различных алгоритмов)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Create dataset
x_data, _ = make_blobs(n_samples=1000, centers=1, cluster_std=1.1, center_box=(0, 0))
plt.scatter(x_data[:, 0], x_data[:, 1], alpha=0.25)
plt.show()

Настроим наш детектор аномалий

In [ ]:
from sklearn.svm import OneClassSVM

svm = OneClassSVM(kernel="rbf", gamma=0.01, nu=0.03)
print(f'SVM parameters:\n{svm}')
svm.fit(x_data)
pred = svm.predict(x_data)

Продемонстрируем аномальные точки предсказанные детектором

In [ ]:
import numpy as np

anom_index = np.where(pred == -1)
values = x_data[anom_index]

plt.scatter(x_data[:, 0], x_data[:, 1], alpha=0.25)
plt.scatter(values[:, 0], values[:, 1], color="r", marker="x")
plt.show()

### Следите за используемыми метриками
Обращайте внимание на то, какие метрики вы используете. При решении задачи классификации часто используется accuracy (точность) равная доли правильно классифицированных объектов. Эта метрика позволяет адекватно оценить результат классификации в случае сбалансированных классов (в наборе одинаковое количество объектов каждого класса). В случае дисбаланса классов данная метрика может выдать обманчиво хороший результат.

Пример: датасет, в котором 95% объектов относятся к классу 1 и 5% к классу 0. Модель, которая для всех данных выдает класс 1, будет иметь $accuracy = 0.95$, хотя данная модель не выдает никакой полезной информации.

Для несбалансированных данных лучше выбирать [MCC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html?highlight=matthews%20correlation%20coefficient#sklearn.metrics.matthews_corrcoef) (Matthews correlation coefficient, коэффициент корреляции Мэтьюса) или [Cohen’s kappa](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) (Коэффициент Каппа Коэна).

<a id='augmentation'></a>
# Аугментация
Другой способ побороть маленькое количество данных для обучения - аугментация. Сам термин пришел из музыки:

**Аугмента́ция** (позднелат. augmentatio — увеличение, расширение) — [техника ритмической композиции в старинной музыке](https://ru.wikipedia.org/wiki/Аугментация).


Модели глубокого обучения обычно требуют большого количества данных для обучения. В целом, чем больше данных, тем лучше для обучения модели. В то же время получение огромных объемов данных сопряжено со своими проблемами (например, с нехваткой размеченных данных или с трудозатратами сопряженными с разметкой).

Вместо того, чтобы тратить дни на сбор данных вручную, мы можем использовать методы аугментации для автоматической генерации новых примеров из уже имеющихся.

Помимо увеличения размеченных датасетов, многие методы *self-supervised learning* построены на использовании разных аугментаций одного и того же сэмпла.


<center><img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/augmentations_examples.png" width="700"></center>
<center><em>Примеры аугментаций картинки. </em></center>

**Важный момент**: при обучении модели мы используем разбиение данных на `train-val-test`. Аугментации стоит применять только на `train`. Почему так? Конечная цель обучения нейросети - это применение на реальных данных, которые сеть не видела. Вот и портить их не надо.

В любом случае, `test` должен быть отделен от данных еще до того как они попали в `DataLoader` или нейросеть.

Другое дело, что аугментации на тесте можно использовать как метод ансамблинга в случае классификации. Можно взять sample -> создать несколько его копий -> по разному их аугментировать -> предсказать класс на каждой из этих аугментированных копий -> а потом выбрать наиболее вероятный класс голосованием (такой функционал реализован, например, в [YOLOv5](https://github.com/ultralytics/yolov5/blob/d204a61834d0f6b2e73c1f43facf32fbadb6b284/models/yolo.py#L121), о которой речь пойдет в следующих лекциях).

## Изображения

Загрузим и отобразим пример картинки. Картинку отмасштабируем, чтобы она не занимала весь экран

In [ ]:
import torch
import random
import numpy as np
# fix random_seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/capybara_image.jpg'
!wget -q $URL -O test.jpg

In [ ]:
from IPython.display import display
from PIL import Image
from torchvision import transforms

input_img = Image.open('/content/test.jpg')
input_img = transforms.Resize(size=300)(input_img) 
display(input_img)

Рассмотрим несколько примеров аугментаций картинок. С полным списком можно ознакомиться на сайте [[doc] документации torchvision](https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py).

### Rotation

Посмотрим, какие параметры принимает на вход `Random Rotation`

In [ ]:
? transforms.RandomRotation

Создадим переменную `transform`, в которую добавим нашу аугментацию и применим ее к исходному изображению. Затем запустим следующую ячейку несколько раз подряд

In [ ]:
import matplotlib.pyplot as plt

transform = transforms.RandomRotation(degrees=(0, 180))

def plot_augmented_img(transform, input_img):
    
    f,ax = plt.subplots(1, 2, figsize=(15, 15))
    augmented_img = transform(input_img)
    ax[0].imshow(input_img)
    ax[0].set_title('Original img')
    ax[0].axis('off')

    ax[1].imshow(augmented_img)
    ax[1].set_title('Augmented img')
    ax[1].axis('off')
    plt.show()

plot_augmented_img(transform, input_img)

### Blur

In [ ]:
? transforms.GaussianBlur

In [ ]:
transform = transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))

plot_augmented_img(transform, input_img)

### Random Erasing

In [ ]:
? transforms.RandomErasing

In [ ]:
transform = transforms.Compose(
    [ 
        transforms.ToTensor(),
        transforms.RandomErasing(p=1),
        transforms.ToPILImage()
    ]
)
    
plot_augmented_img(transform, input_img)

### ColorJitter

In [ ]:
? transforms.ColorJitter

In [ ]:
transform = transforms.ColorJitter(brightness=0.5, hue=0.3)

plot_augmented_img(transform, input_img)

### Совмещаем несколько аугментаций вместе

Для этого будем использовать метод `Compose`. Нам нужно будет создать `list` со всеми аугментациями, которые будут применены последовательно.

In [ ]:
transform = transforms.Compose(
    [
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5, p=1.0),
        transforms.ColorJitter(brightness=0.5, hue=0.3)
    ]
)

plot_augmented_img(transform, input_img)

### А что если мы хотим применять аугментации случайным образом?

Для этого воспользуемся методом `RandomApply`, который на вход принимает метод аугментации, и вероятность `p`, что эта аугментация будет применена.

In [ ]:
? transforms.RandomApply

In [ ]:
transform = transforms.RandomApply(
    transforms=[
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5),
        transforms.ColorJitter(brightness=0.5, hue=0.3)
    ],
    p=0.7
)


plot_augmented_img(transform, input_img)

### Аугментация внутри `Dataset`

Возьмем папку с картинками

In [ ]:
import os
from zipfile import ZipFile
from IPython.display import clear_output

os.chdir('/content')
# download files
!wget --no-check-certificate 'https://edunet.kea.su/repo/EduNet-web_dependencies/L11/for_transforms.Compose.zip' -O data.zip
with ZipFile('data.zip', 'r') as folder: # Create a ZipFile Object and load sample.zip in it
    folder.extractall() # Extract all the contents of zip file in current directory
clear_output()

In [ ]:
os.chdir("/content/for_transforms.Compose")
img_list = os.listdir()
print(img_list)

Напишем класс `Dataset`

In [ ]:
from torch.utils.data import Dataset

class AugmentationDataset(Dataset):
    def __init__(self, img_list, transforms=None):
        self.img_list = img_list
        self.transforms = transforms

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, i):
        img = plt.imread(self.img_list[i])
        img = Image.fromarray(img).convert("RGB")
        img = np.array(img).astype(np.uint8)

        if self.transforms is not None:
            img = self.transforms(img)
        return img 

Напишем вспомогательную функцию для отображения картинок

In [ ]:
def show_img(img):
    plt.figure(figsize=(40, 38))
    img_np = img.numpy()
    plt.imshow(np.transpose(img_np, (1, 2, 0)))
    plt.show()

Создадим `list` с аугментациями, которые мы хотим применить. Чтобы загрузить аугментации в `PyTorch`, нам необходимо эти картинки преобразовать в тензоры. Для этого воспользуемся родным торчевым преобразованием `transforms.ToTensor()`

In [ ]:
tensor_transform = transforms.Compose(
    [   
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5),
        transforms.ToTensor()
    ]
)

Теперь обернем все в `DataLoader` и отобразим

In [ ]:
from torch.utils.data import DataLoader
import torchvision

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, tensor_transform), batch_size=8, shuffle=True)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

Существует и более сложные способы аугментации:

- **Mixup**

Mixup можно представить с помощью этого простого уравнения:

`newImage = alpha * image1 + (1-alpha) * image2`

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/mixup_augmentation_scheme.png" width="700">

Подробнее в статьях:

[mixup: Beyond Empirical Risk Minimization](https://arxiv.org/abs/1710.09412)

[On Mixup Training](https://proceedings.neurips.cc/paper/2019/file/36ad8b5f42db492827016448975cc22d-Paper.pdf)



- **Аугментация при помощи генерации данных**

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/augmentation_using_data_synthesis.png" width="600">

Кроме методов, реализованных в Pytorch, существуют и специализированные библиотеки для аугментации изображений. 

Например:
- [Albumentations](https://albumentations.ai)
- [imgaug](https://imgaug.readthedocs.io/en/latest/index.html)
- [AugLy](https://github.com/facebookresearch/AugLy)

**При выборе методов аугментации имеет смысл использовать те, которые будут в реальной жизни. Например, нет смысла делать перевод изображения в черно-белое, если предполагается, что весь входящий поток будет цветным, или отражать человека вверх-ногами, если мы не предполагаем его таким распознавать.**

## Аудио

Рассмотрим несколько примеров аугментаций аудио. С полным списком можно ознакомиться здесь [[git] audiomentations](https://github.com/iver56/audiomentations).

Импортируем библиотеку и посмотрим на пример

In [ ]:
os.chdir('/content')

!pip install audiomentations

!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L11/audio_example.wav
clear_output()

In [ ]:
from IPython.display import Audio

# Get input audio
input_audio ='/content/audio_example.wav'

display(Audio(input_audio))

In [ ]:
import librosa

data, sr = librosa.load('/content/audio_example.wav') # sr - sampling rate

### Background Noise


In [ ]:
from audiomentations import AddGaussianSNR

augment = AddGaussianSNR(min_snr_in_db=3, max_snr_in_db=7, p=1)

# Augment/transform the audio data
augmented_data = augment(samples=data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

Сравним волновые картины и спектрограммы

In [ ]:
from scipy.signal import spectrogram

def produce_plots(input_audio_arr, aug_audio, sr):
    f,t, Sxx_in = spectrogram(input_audio_arr, fs=sr) # Compute spectrogram for the original signal (f - frequency, t - time)
    f,t, Sxx_aug = spectrogram(aug_audio, fs=sr)

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 5)) 

    ax[0,0].plot(input_audio_arr)
    ax[0,0].set_xlim(0, len(input_audio_arr))
    ax[0,0].set_xticks([])
    ax[0,0].set_title('Original audio')

    ax[0,1].plot(aug_audio)
    ax[0,1].set_xlim(0, len(input_audio_arr))
    ax[0,1].set_xticks([])
    ax[0,1].set_title('Augmented  audio')

    ax[1,0].imshow(np.log(Sxx_in), 
                   extent=[t.min(), t.max(), f.min(), f.max()],
                   aspect='auto',
                   cmap='inferno')
    ax[1,0].set_ylabel('Frequecny, Hz')
    ax[1,0].set_xlabel('Time,s')
    
    ax[1,1].imshow(np.log(Sxx_aug), 
                   extent=[t.min(), t.max(), f.min(), f.max()],
                   aspect='auto',
                   cmap='inferno')
    ax[1,1].set_ylabel('Frequecny, Hz')
    ax[1,1].set_xlabel('Time,s')

    plt.subplots_adjust(hspace=0)
    plt.show()

produce_plots(data, augmented_data, sr)

### Time Stretch

In [ ]:
from audiomentations import TimeStretch

augment = TimeStretch(min_rate=0.8, max_rate=1.5, p=1) 
augmented_data = augment(data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

In [ ]:
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

produce_plots(data, augmented_data, sr)

### Pitch Shift

Изменение тональности:

In [ ]:
from audiomentations import PitchShift

augment = PitchShift(min_semitones=1, max_semitones=12, p=1)
augmented_data = augment(data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

### Совмещаем несколько аугментаций вместе

Как и в случае с картинками мы можем совмещать несколько аугментаций вместе

In [ ]:
from audiomentations import Compose, AddGaussianNoise, Shift

augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=1),
    PitchShift(min_semitones=-4, max_semitones=4, p=1),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=1),
])

augmented_data = augment(data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

Посмотрим на то, что получилось

In [ ]:
produce_plots(data, augmented_data, sr)

Дополнительные библиотеки для аугментации звука (и волновых функций в целом):
- [torchaudio](https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html)
- [torch-audiomentations](https://github.com/asteroid-team/torch-audiomentations)
- [AugLy](https://github.com/facebookresearch/AugLy)

## Текст

Теперь рассмотрим несколько примеров аугментаций текста. С полным списком можно ознакомиться здесь [[git] сайте библиотеки](https://github.com/makcedward/nlpaug).

In [ ]:
!pip install -q nlpaug

In [ ]:
# Define input text
text = "Hello, future of AI for Science! How are you today?"
print(f'input text: {text}')

### Аугментация символов

Заменой на похоже выглядящие:

In [ ]:
import nlpaug.augmenter.char as nac

augment = nac.OcrAug()
augmented_text = augment.augment(text)

print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

С опечатками, которые учитывают расположение символов на клавиатуре:

In [ ]:
augment = nac.KeyboardAug()
augmented_text = augment.augment(text)

print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

### Аугментация слов

С орфографическими ошибками:

In [ ]:
import nlpaug.augmenter.word as naw

augment = naw.SpellingAug()
augmented_text = augment.augment(text, n=3)

print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

С использованием модели для предсказания новых слов в зависимости от контекста:

In [ ]:
!pip install -q transformers

In [ ]:
from IPython.display import clear_output
# model_type: word2vec, glove or fasttext
augment = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
augmented_text = augment.augment(text)

clear_output()
print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

Мы можем переводить текстовые данные на какой-либо язык, а затем перевести их обратно на язык оригинала. Это может помочь сгенерировать текстовые данные с разными словами, сохраняя при этом контекст текстовых данных.

In [ ]:
!pip -q install sacremoses

In [ ]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)
augmented_text = back_translation_aug.augment(text)

clear_output()
print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

Дополнительные  библиотеки для аугментации текста:


- [TextAugment](https://github.com/dsfsi/textaugment)
- [AugLy](https://github.com/facebookresearch/AugLy)


# Transfer Learning
----
Как обучить нейросеть на своих данных, когда их мало?

Для таких типовых задач, как классификация изображений, можно воспользоваться одной из существующих архитектур (AlexNet, VGG, Inception, ResNet и т.д.) и просто обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что на данном этапе можно использовать одну из них как черный ящик, не вникая в принцип её работы. Например, в PyTorch есть много уже реализованных известных архитектур: [torchvision.models](https://pytorch.org/vision/stable/models.html).


Однако, глубокие нейронные сети требуют большие объемы данных для успешного обучения. И зачастую, в нашей частной задаче недостаточно данных для того, чтобы хорошо обучить нейросеть с нуля. `Transfer learning` решает эту проблему. По сути, мы пытаемся использовать опыт, полученный нейронной сетью при обучении на некоторой задаче $T_1$, чтобы решать схожую задачу $T_2$.

К примеру, `transfer learning` можно использовать при решении задачи классификации изображений на небольшом наборе данных. Как уже ранее обсуждалось, при обработке изображений свёрточные нейронные сети в первых слоях "реагируют" на некие простые пространственные шаблоны (к примеру, углы), после чего комбинируют их в сложные осмысленные формы (к примеру, глаза или носы). Вся эта информация извлекается из изображения, создавая новые сложные представления данных, в результате классифицируемые линейной моделью. 

Идея заключается в том, что если изначально обучить модель на некоторой сложной и довольно общей задаче, можно надеяться что она (как минимум часть ее слоев), в общем, случае будет извлекать важную информацию из изображений, и полученные представления можно будет успешно использовать для классификации линейной моделью.

Таким образом, берем часть модели, которая, по нашему представлению, отвечает за выделение хороших признаков (часто - все слои, кроме последнего) - feature extractor. Присоединяем к это части дополнительные слой/cлои для решения уже новой задачи. И учим только эти слои. Cлои feature extractor не учим - они "заморожены"

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_change_classes_scheme.png" width="700">

Понятно, что не все фильтры модели будут использованы максимально эффективно - к примеру, если мы работаем с изображениями, связанными с едой, возможно, не все фильтры на скрытых слоях предобученной на ImageNet модели окажутся полезны для нашей задачи. Почему бы не попробовать **не только обучить новый классификатор, но и дообучить некоторые промежуточные слои**? При использовании этого подхода мы при обучении дополнительно "настраиваем" и промежуточные слои, называется он `fine-tuning`.

В дальнейшем примере мы вообще не будем фиксировать веса - этот вариант, по сути, также относится к `fine-tuning`. В таком подходе learning rate ставится ниже, чем при обучении нейросети с нуля - мы знаем, что, по крайней мере, часть весов нейросети выполняют свою задачу хорошо, и не хотим испортить это быстрыми первыми изменениями. 

Кроме этого, можно делать комбинации этих методов - сначала учить только последние, добавленные нами слои сети. Затем учить еще и самые близкие к ним. Затем учить уже все веса нейросети вместе. То есть мы можем определить свою **стратегию fine-tuning**. 

Иногда **fine-tuning** считается синонимом "transfer learning", в этом случае часть от предтренированной сети называют **backbone**, а добавленную часть - **head**.  Подробнее про это можно прочитать [здесь](https://lightning-flash.readthedocs.io/en/latest/general/finetuning.html)

 

## Структурные компоненты

Для выполнения данной задачи нам понадобятся несколько компонент.

1. В первую очередь, нам необходима сама обученная модель, которую мы будем адаптировать под нашу локальную задачу. Модель можно получить из фреймворка, либо скачать по ссылке, приложенной авторами интересующей Вас статьи. 

2. Также, как и при работе с обычной моделью, понадобится кодовая база для обучения, включающая в себя известные шаги: подсчёт градиента, шаг оптимизатора, подсчёт качества на проверочной выборке и т.п.  

3. После того как обучение будет выполнено, понадобится использовать модель в реальных условиях, либо как минимум проверить её качество на тестовой выборке. В данном случае, нам понадобится кодовая база для выполнения предсказаний. 

4. В некоторых случаях, веса модели находятся отдельно от описания (архитектуры) модели, в таком случае необходимо дополнительно озаботиться их получением и загрузкой в модель.  

5. Последняя важная компонента - данные. Необходимо тщательно подготовить данные, на которых будет дообучаться модель, проверить разбиение на обучающую, проверяющую и тестировочную выборки, чтобы избежать утечек данных и т.п.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/structural_components_scheme.png" width="750">

## Практический пример Transfer Learning

Давайте рассмотрим пример практической реализации такого подхода ([код переработан из этой статьи](https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/)).

 Загрузим датасет и удалим из него 90% файлов



In [ ]:
import os
from IPython.display import clear_output
from random import sample

!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L11/EuroSAT.zip # http://madm.dfki.de/files/sentinel/EuroSAT.zip
!unzip EuroSAT.zip

os.chdir('/content')
path = '/content/2750/'

for folder in os.listdir(path):
  files = os.listdir(path+folder)
  for file in sample(files,int(len(files)*0.9)):
      os.remove(path+folder+'/'+file)
      
clear_output()

Определим аугментации. Для разнообразия будем использовать родные аугментации из библиотеки PyTorch Vision

In [ ]:
import torch
import random
import numpy as np
from torchvision import transforms
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Applying Transforms to the Data
img_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize(size=128),
            transforms.RandomRotation(degrees=15),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
        ]
    ),
    # No augmentations on valid data!
    "valid": transforms.Compose( 
        [
            transforms.Resize(size=128),
            transforms.ToTensor(),
        ]
    ),
    # No augmentations on test data!
    "test": transforms.Compose(
        [
            transforms.Resize(size=128),
            transforms.ToTensor(),
        ]
    ),
}

Создадим `datasets`

In [ ]:
from torchvision import datasets
from copy import deepcopy

dataset = datasets.ImageFolder(root=path)
# split to train/valid/test
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.1), int(len(dataset)*0.1)])

train_set.dataset = deepcopy(dataset)
valid_set.dataset = deepcopy(dataset)
test_set.dataset = deepcopy(dataset)

# define augmentations
train_set.dataset.transform = img_transforms['train']
valid_set.dataset.transform = img_transforms['valid']
test_set.dataset.transform = img_transforms['test']

print(f'Train size: {len(train_set)}')
print(f'Valid size: {len(valid_set)}')
print(f'Test size: {len(test_set)}')

In [ ]:
from torch.utils.data import DataLoader
# Batch size
batch_size = 64

# Number of classes
num_classes = len(dataset.classes)

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size, valid_data_size = len(train_set), len(valid_set)

# Create iterators for the Data loaded using DataLoader module
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
print('indexes to class: ')
idx_to_class

В наборе данных не так уж и много изображений. При обучении с нуля нейросеть скорее всего не достигнет высокой точности.

Загрузим AlexNet без весов и попробуем обучить с нуля

In [ ]:
from torchvision import models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

alexnet = models.alexnet(weights=None)
print(alexnet)

Затем мы заменяем последний слой модели AlexNet слоем с `num_classes` нейронами, в соответствии с числом классов в нашем подмножестве.

То есть мы "сказали" нашей модели распознавать не `1000`, а только `num_classes` классов.

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
import torch.nn as nn

alexnet.classifier[6] = nn.Linear(4096, num_classes) # change out classes, from 1000 to 10
alexnet.classifier.add_module("7", nn.LogSoftmax(dim=1)) # add activation
print(alexnet)

Затем мы определяем функцию потерь и оптимизатор, который будет использоваться для обучения.

In [ ]:
import torch.optim as optim

# Define Optimizer and Loss Function
criterion = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=3e-4)
print(optimizer)

Для тренировки и валидации нашей модели напишем отдельную функцию.

In [ ]:
import time

def train_and_validate(model, criterion, optimizer, num_epochs=25, save_state=False):
    """
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)

    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    """

    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, num_epochs))

        # Set to training mode
        model.train()

        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0

        valid_loss = 0.0
        valid_acc = 0.0

        train_correct = 0
        for i, (inputs, labels) in enumerate(train_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Clean existing gradients
            outputs = model(inputs)  # Forward pass - compute outputs on input data using the model
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagate the gradients
            optimizer.step()  # Update the parameters

            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            # Compute correct predictions
            train_correct += (torch.argmax(outputs, dim=-1) == labels).float().sum()

        # Compute the mean train accuracy
        train_accuracy = 100 * train_correct / (len(train_loader) * batch_size)

        val_correct = 0
        # Validation - No gradient tracking needed
        with torch.no_grad():

            model.eval()  # Set to evaluation mode

            # Validation loop
            for j, (inputs, labels) in enumerate(valid_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)  # Forward pass - compute outputs on input data using the model
                loss = criterion(outputs, labels)  # Compute loss
                valid_loss += loss.item() * inputs.size(0)  # Compute the total loss for the batch and add it to valid_loss

                val_correct += (torch.argmax(outputs, dim=-1) == labels).float().sum()

        # Compute mean val accuracy       
        val_accuracy = 100 * val_correct / (len(valid_loader) * batch_size)

        # Find average training loss and training accuracy
        avg_train_loss = train_loss / (len(train_loader) * batch_size)

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss / (len(valid_loader) * batch_size)

        history.append([avg_train_loss, avg_valid_loss,train_accuracy.detach().cpu(), val_accuracy.detach().cpu()])

        epoch_end = time.time()

        print(
            "Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
                epoch + 1,
                avg_train_loss,
                train_accuracy.detach().cpu(),
                avg_valid_loss,
                val_accuracy.detach().cpu(),
                epoch_end - epoch_start,
            )
        )
        #Saving state for fine_tuning (because we may overfit)
        if save_state:
          os.makedirs('check_points', exist_ok=True)
          torch.save(model.state_dict(), f'check_points/fine_tuning_{epoch + 1}.pth')

    return model, history

Теперь обучим нашу модель

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    alexnet.to(device), criterion, optimizer, num_epochs
)

torch.save(history, "history_fresh.pt")

Посмотрим на графики

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
fig.suptitle("Fresh learning", fontsize=14)

history = np.array(history)
ax[0].plot(history[:, :2])
ax[0].legend(["Train Loss", "Val Loss"])
ax[1].plot(history[:, 2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Результаты оставляют желать лучшего.

Теперь давайте попробуем использовать *transfer learning*. 

Загрузим предобученную модель Alexnet:

In [ ]:
del alexnet
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT')

В данном случае, мы не дообучаем скрытые слои нашей модели, потому отключаем подсчёт градиентов ("**замораживаем**" параметры). Данный шаг гарантирует, что обучение (изменение параметров) будет происходить только на последнем слое модели.

In [ ]:
# Freeze model parameters
for param in alexnet.parameters():
    param.requires_grad = False

Меняем последний слой таким образом, чтобы он вместо 1000 классов, ImageNet нам выдавал количество классов, которое у нас есть

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
alexnet.classifier[6] = nn.Linear(4096, num_classes)
alexnet.classifier.add_module("7", nn.LogSoftmax(dim=1))

In [ ]:
# Define Optimizer and Loss Function
criterion = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=3e-4)

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    alexnet.to(device), criterion, optimizer, num_epochs
)

torch.save(history, "history_transfer_learning.pt")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
fig.suptitle("Transfer learning", fontsize=14)

history = np.array(history)
ax[0].plot(history[:, :2])
ax[0].legend(["Train Loss", "Val Loss"])
ax[1].plot(history[:, 2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

И сравним между собой

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
fig.suptitle("Fresh Learning (FL) vs Transfer Learning (TL)", fontsize=14)

history_fresh = np.array(torch.load('history_fresh.pt'))
history_transfer_learning = np.array(torch.load('history_transfer_learning.pt'))

ax[0].plot(history_fresh[:, :2])
ax[0].plot(history_transfer_learning[:, :2])
ax[0].legend(["Train Loss (FL)", "Val Loss (FL)", "Train Loss (TL)", "Val Loss (TL)"])

ax[1].plot(history_fresh[:,2:])
ax[1].plot(history_transfer_learning[:,2:])
ax[1].legend(["Train Accuracy (FL)", "Val Accuracy (FL)", "Train Accuracy (TL)", "Val Accuracy (TL)"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Определенно стало лучше =)

Проведём процедуру **fine-tuning**. 
В предыдущем варианте с Transfer Learning обучались только последние слои, добавленные вручную. Давайте проверим это, выведя те слои, в которых включён градиент.

In [ ]:
for name, param in alexnet.named_parameters(): 
    print(name, param.requires_grad)

Мы оставим дообученную голову нейронной сети и продолжим **обучение всей сети с уменьшением темпа обучения**. 

**Разморозим** параметры. `Criterion` остаётся тот же, в `optimizer` уменьшим параметр `lr` на порядок.

In [ ]:
for param in alexnet.parameters():
    param.requires_grad = True

optimizer = optim.Adam(alexnet.parameters(), lr=3e-5)

Пройдём дополнительные 20 эпох и построим графики.

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    alexnet.to(device), criterion, optimizer, num_epochs, save_state=True
    )

torch.save(history, "history_finetuning.pt")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(16, 5))
fig.suptitle("Transfer Learning (TL) AND Finetuning (FT)", fontsize=14)

history_transfer_learning = np.array(torch.load('history_transfer_learning.pt'))
history_finetuning = np.array(torch.load('history_finetuning.pt'))

train_val_loss = np.concatenate((history_transfer_learning[:, :2], history_finetuning[:, :2]), axis=0)
ax[0].plot(train_val_loss)
ax[0].legend(["Train Loss", "Val Loss"])

ax[0].vlines(20, 0, 1.5,
          color='g',
          linewidth=5,
          linestyle='--')

train_val_acc = np.concatenate((history_transfer_learning[:, 2:], history_finetuning[:, 2:]), axis=0)

ax[1].plot(train_val_acc)
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[1].vlines(20, 0, 100,
          color='g',
          linewidth=5,
          linestyle='--')

ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Есть ли эффект от **fine-tuning**?. После дообучения ещё на 20 эпохах мы наблюдаем следующие эффекты:


*   На валидации: Loss и accuracy вышли на плато, ошибка уменьшилась, точность возросла.
*   На обучении: Loss стремится к 0, тогда как точность к 100%. Началось **переобучение** примерно с 25 эпохи. Fine-tuning склонен к переобучению (этого можно попробовать избежать, делая "голову" нейросети многослойной, плавно уменьшая количество нейронов), следите за метриками и ошибкой на тренировочной выборке.
*   В промежутке эпох 20-25 мы **добились заметного улучшения** работы модели.



Ожидая переобучения, мы сохранили состояния нейросети на каждой эпохе. Возьмём состояние с 24 эпохи как наиболее оптимальное.

In [ ]:
trained_model.load_state_dict(torch.load('check_points/fine_tuning_4.pth')) # 20 + 4
trained_model.eval()

Посмотрим на картинки

In [ ]:
def predict(model, test_img_name, device):
    """
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_img_name: Test image

    """

    transform = img_transforms["test"]
    test_img = torch.tensor(np.asarray(test_img_name)) 
    test_img = transforms.ToPILImage()(test_img)
    plt.imshow(test_img)

    test_img_tensor = test_img_name.unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_img_tensor).to(device)
        ps = torch.exp(out).to(device)
        topk, topclass = ps.topk(3, dim=1)
        for i in range(3):
            print(
                "Predcition",
                i + 1,
                ":",
                idx_to_class[topclass.cpu().numpy()[0][i]],
                ", Score: ",
                round(topk.cpu().numpy()[0][i], 2),
            )

In [ ]:
print("Shoud be %s\n" % idx_to_class[0])
predict(
    trained_model.to(device),
    valid_set[[np.where([x[1] == 0 for x in valid_set])[0]][0][1]][0],
    device,
)

In [ ]:
print("Shoud be %s\n" % idx_to_class[6])
predict(
    trained_model,
    valid_set[[np.where([x[1] == 6 for x in valid_set])[0]][0][10]][0],
    device,
)

In [ ]:
print("Shoud be %s\n" % idx_to_class[8])
predict(
    trained_model,
    valid_set[[np.where([x[1] == 8 for x in valid_set])[0]][0][5]][0],
    device,
)

* Мы увидели, как использовать предварительно обученную модель на 1000 классов ImageNet для нашей задачи на 10 классов.

* Мы сравнили качество **обучения с нуля, transfer learning и fine-tuing** и научились добиваться максимального качества с помощью этих принципов.

* И столкнулись с характерной **опасностью fine-tuning - переобучением**. Используйте **низкий learning rate** и отслеживайте Loss - возможно, вам будет достаточно **небольшого количества эпох**.

# Few/One-Shot learning

Существуют задачи, где не представляется возможным разбить данные на классы так, чтобы в каждом классе было достаточно много объектов.

Рассмотрим, например, задачу распознавания лиц.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/videoanalytics.png"  width="800">

На вход системе подается фото лица человека. Требуется сопоставить его с другим изображениям(и) например хранящимися в БД и таким образом идентифицировать человека на фотографии.

На первый взгляд кажется, что это задача классификации.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/classifier_scheme.png"  width="700">


Все изображения одного человека будем считать относящимися к одному классу, и модель будет этот класс предсказывать. 

Для небольшой организации, в которой всего несколько десятков сотрудников такой подход может сработать.
При этом возникнут проблемы:

1. Чтобы обучить такую ​​систему, нам сначала потребуется много (сотни) разных 
изображений каждого сотрудника.

2. Когда человек присоединяется к организации или покидает ее, придется поменять структуру модели и обучать ее заново.


Это практически невозможно, для крупных организаций, где набор и увольнение происходит почти каждую неделю. И в принципе невозможно для города масштаба Москвы или Лондона, в котором миллионы жителей и сотни тысяч приезжих.

## Формирование векторов-признаков (embedding)

Поэтому используется другой подход.
Вместо того что бы классифицировать изображения, модель учится выделять ключевые признаки и на их основе строить компактный вектор достаточно точно описывающий лицо.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/face_as_embedding.png"  width="500">




В англоязычной литературе такие вектора признаков называются **embedding** и мы тоже будем использовать это обозначение.





Может возникнуть вопрос: не потеряем ли мы важную информацию, сжав изображение в несколько сотен чисел? 

Чтобы ответить на него вспомним, как работает [фоторобот](https://en.wikipedia.org/wiki/Facial_composite). 


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/photorobot.png"  width="700">



Для получения фотореалистичного изображения лица достаточно нескольких ключевых признаков: Глаза, волосы рот, нос ...

Каждый, из которых кодируется максимум несколькими сотнями целых значений. 

Значит вектора - признака из 128 вещественных чисел будет более чем достаточно.
Правда интерпретировать значения, которые закодирует в него нейросеть, будет не столь просто.


Если нам удастся обучить модель кодировать в embedding признаки важные для сравнения, то мы сможем сравнивать вектора между собой.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/face_dist.png"  width="700">



Если расстояние между векторами для лиц, которые похожи друг на друга будут маленькими, а у непохожих наоборот большими. То мы сможем экспериментально подобрать порог $d$ и, сравнивая с ним расстояние между двумя векторами, принимать решение: принадлежат ли они одному человеку или нет.






*Можно оценивать не расстояние, а степень схожести similarity. В этом случае неравенства поменяют знак, но логика останется прежней*




Теперь, чтобы идентифицировать человека, требуется только одно изображение его лица. Эмбеддинг  этого изображения можно сравнить с эмбеддингами других лиц из БД используя  [KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) или иной метод кластеризации.



<img src ="http://edunet.kea.su/repo/EduNet-content/L11/out/search_in_embedding_space.jpg"  width="700">



Такая модель не учится классифицировать изображение напрямую по какому-либо из выходных классов. Она учится выделять признаки, важные при сравнении.

Такой подход решает обе проблемы, о которых мы говорили выше:
- Для обучения такой сети нам не требуется много экземпляров объектов одного класса, а   достаточно лишь нескольких.
- Но самое большое преимущество в простоте ее обучения в случае появления новых объектов.

## Сиамская сеть (Siamese Network)

Какая архитектура должна быть у модели генерирующей вектора-признаков?

Можно было бы использовать обычную сеть, обученную для задачи классификации, и затем удалить из нее последний(е) слой.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/embedding_from_classifier.png"  width="800">



Активации последнего слоя представляют собой отклики на некие высокоуровневые признаки потенциально важные для классификации и их можно интерпретировать как embedding.







In [ ]:
from torchvision.models import alexnet
import torch

face1 = torch.randn((3, 224, 224))
face2 = torch.randn((3, 224, 224))

model = alexnet(weights='AlexNet_Weights.DEFAULT')
# remove classification layer 
model.fc = model.classifier[6] = torch.nn.Identity()

# get embeddings
embedding1 = model(face1.unsqueeze(0)) 
embedding2 = model(face2.unsqueeze(0)) 

diff = torch.nn.functional.pairwise_distance(embedding1, embedding2 )
print("L2 distance: ", diff.item())

Такой подход будет работать.
Однако можно заметно улучшить точность, используя лосс-функцию, которая оценивает именно качество сравнения, а не классификации.


Рассмотрим подход, основанный на методологии, описанной в статье [Siamese Neural Networks for One-shot Image Recognition (Koch et al., 2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf).




<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/siamese_neural_network_scheme.png"  width="800">


<center><em>Используются две копии одной и той же сети, отсюда и название Siamese Networks.</em></center>

Два входных изображения ($x_1$ и $x_2$) проходят через одну и ту же сверточную сеть, на выходе для каждого изображения генерируется вектор признаков фиксированной длины $h_{x_{1}}$ и $h_{x_{2}}$.






Модель обучается генерировать близкие вектора для изображений одного объекта и далекие для разных.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/siamese_neural_network_idea_scheme.png"  width="700">


Оценивая расстояние между двумя векторами признаков 
которое будет малым для одних и тех же объектов и большим для различных мы сможем оценить их сходство.

Это центральная идея сиамских сетей.

### Triplet Loss

Какой loss использовать для обучения такой сети?

Очевидно loss функция должна будет учитывать не один выход как минимум два.

Популярной на сегодняшний день является  `Triplet loss`, которой требуется  три embedding вместо двух. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/triplet_loss_scheme.png"  width="700">


Чтобы сгенерировать три эмбеддинга, модель должна получать на вход три изображения. 

Первые два должны относиться к одному и тому же объекту (человеку), а третье к другому.


Таким образом, Триплет состоит из базового ("якорного" `anchor`), положительного(`positive`) и отрицательного(`negative`) образцов.

Описание в статье [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/abs/1503.03832)



Сама лосс функция будет выглядеть следующим образом:

$$TripletLoss = \sum_{1}^{N} L_i(x_i^{a},x_i^{p},x_i^{n})$$

$$L_i(x_i^{a},x_i^{p},x_i^{n})=max(0,\left\| f(x_i^{a}) -f(x_i^{p}) \right\|_2^{2} - \left\| f(x_i^{a}) -f(x_i^{n}) \right\|_2^{2} + margin)$$

Где:


$x_i^{a}$ - базовое изображение (anchor)

$x_i^{p}$ - изображение того же объекта (positive)

$x_i^{n}$ - изображение другого объекта (negative)

$f(x)$ - **нормированный** выход модели (embedding) для входа $x$


$\left\| x \right\|_2$ -это L2 (Euclidean norm), соответственно $\left\| a \right\|_2^{2}$ это L2 в квадрате.

$margin$ - это константа или минимальный 'зазор', на который расстояние до эмбеддинга негативного объекта обязано превосходить расстояние до позитивного (идея такая же, что в SVM Loss) .







<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/triplet_loss_idea_scheme.png"  width="900">

В ходе обучения с  Triplet loss расстояние между эмбеддингами опорного объекта и позитивного уменьшается, а для отрицательного увеличивается.


Важным дополнением является то, что embedding-и нормируются. В результате нормировки каждый вектор-признак будет иметь единичную длину.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/sphere_distance.png"  width="300">



Теперь мы можем рассматривать embedding-и как точки на n-мерной сфере с радиусом 1. 

Это удобно, так как все расстояния между embedding будут лежать в интервале [0..2] и нам будет проще подобрать порог для сравнения.

Кроме того, можно использовать другие меры расстояния, например [косинусное расстояние](https://buildwiki.ru/wiki/Cosine_similarity), которое определяется углом между векторами и лежит в интервале [-1 ..1] и будет соответствовать расстоянию между точками на поверхности сферы.




В статье авторы минимизируют Евклидово расстояние, но подход будет работать и для других метрик сходства, например косинусного расстояния.

В Pytorch есть две реализации TripletLoss

[TripletMarginLoss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html) - минимизирует $L_p$ норму 

In [ ]:
from torch import nn

triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
anchor = torch.randn(100, 128, requires_grad=True)
positive = torch.randn(100, 128, requires_grad=True)
negative = torch.randn(100, 128, requires_grad=True)
loss = triplet_loss(anchor, positive, negative)
print(loss)

[TripletMarginWithDistanceLoss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginWithDistanceLoss.html#torch.nn.TripletMarginWithDistanceLoss) позволяет задать произвольную функцию расстояния.

In [ ]:
import torch.nn.functional as F
 
triplet_loss = nn.TripletMarginWithDistanceLoss( 
      margin=1,
      distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y)
    )
loss = triplet_loss(anchor, positive, negative)
print(loss)

Другие Лосс - функции для сиамских сетей:

Исторически первой появилась `Contrastive Loss`, о ней подробнее в статье [Dimensionality Reduction by Learning an Invariant Mapping (Hadsell et al., 2005)](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)


В Pytorch есть реализация 
[CosineEmbeddingLoss](https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html), она позволяет обучать модель на парах изображений, минимизировав [косинусное расстояние](https://buildwiki.ru/wiki/Cosine_similarity) между embedding.


## Реализация сиамской сети


### Загрузка данных

Загрузим небольшой фрагмент датасета с лицами. Внутри архива фото лиц сгруппированы по папкам



*   s1/
**  photo1.pgm
**  photo2.pgm
**  ...
*   s2/
*   s2/
*    ...
*   sn/

В каждой папке фото лица одного и того же человека.

In [ ]:
from IPython.display import clear_output
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L11/small_face_dataset.zip
!unzip small_face_dataset.zip
clear_output()

Чтобы результаты воспроизводились, зафиксируем SEED

In [ ]:
import os
import numpy as np
import random

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)


set_random_seed(42)

### Dataset for TripletLoss

Для TripletLoss потребуются три изображения: anchor, positive, negative и метод __get_item__ должен возвращать их нам. Первые два должны принадлежать одному человеку, а третье другому. 



In [ ]:
from torch.utils.data import Dataset
from glob import glob
from PIL import Image

class SiameseNetworkDataset(Dataset):
    
    def __init__(self, dir=None, transform=None, splitter='/'):
        self.dir = dir
        self.splitter = splitter
        self.transform = transform        
        self.files = glob(f"{self.dir}/**/*.pgm",recursive=True)
        self.data =self.build_index()        
    
    def build_index(self):
      index = {}      
      for f in self.files:
        id = self.path2id(f) 
        if not id in index:
          index[id] = []
        index[id].append(f)
      return index
    
    def path2id(self, path):
        return path.replace(self.dir, "").split(self.splitter)[0]

    def __getitem__(self, index):
        anchor_path = self.files[index]
        positive_path = self.find_positive(anchor_path)
        negative_path = self.find_negative(anchor_path)
        
        # Loading the images
        anchor = Image.open(anchor_path)
        positive = Image.open(positive_path)
        negative = Image.open(negative_path)
                
        if self.transform is not None:  # Apply image transformations           
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

    def find_positive(self, path):
        id = self.path2id(path) 
        all_exept_my = self.data[id].copy()
        all_exept_my.remove(path)
        return random.choice(all_exept_my)

    def find_negative(self, path):
        all_exept_my_ids = list(self.data.keys())
        id = self.path2id(path) 
        all_exept_my_ids.remove(id)
        selected_id = random.choice(all_exept_my_ids)
        return random.choice(self.data[selected_id])
    
    def __len__(self):
        return len(self.files)

Выведем несколько изображений, чтобы убедиться, что класс датасета функционирует должным образом.

In [ ]:
from torch.utils.data import  DataLoader
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# Create dataset instance
siamese_dataset = SiameseNetworkDataset("faces/training/",
                                          transform=transforms.Compose([
                                            transforms.Resize((105, 105)),
                                            transforms.ToTensor(),
                                          ])
                                        )

# Create dataloader & extract batch of data from it
vis_dataloader = DataLoader(siamese_dataset, batch_size=8, shuffle=True)
dataiter = iter(vis_dataloader)
example_batch = next(dataiter) # anc, pos, neg

# Show batch contents 
concatenated = torch.cat((example_batch[0], example_batch[1], example_batch[2]), 0)
grid = torchvision.utils.make_grid(concatenated)

plt.axis("off")
plt.imshow(grid.permute(1, 2, 0).numpy())
plt.gcf().set_size_inches(20, 60)
plt.show()    

В каждом столбце тройка изображений. Первое и второе принадлежат одному человеку, третье - другому.

### Создание модели

Нас устроит любая модель для работы с изображениями. Например, Resnet18. 

Все что от нас требуется это:
- заменить последний слой
- отправлять на анализ три изображения вместо одного. Соответственно на выходе тоже будут три вектора признаков (embedding)


Пожалуй, единственный вопрос это размерность последнего слоя. В промышленных системах распознавания лиц, которые тренируются на датасетах из миллионов изображений, используются embedding размерностью от 128 до 512.

Значит, для демонстрационной задачи нам с запасом хватить 32 значений. Значит,  выход последнего линейного слоя должен быть равен 32.

In [ ]:
from torchvision.models import resnet18

class SiameseNet(nn.Module):

    def __init__(self, latent_dim):
        super().__init__()
        self.model = resnet18(weights=None)
        # Because we use grayscale images reduce input channel count to one
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # Replace ImageNet 1000 class classifier with 64- out linear layer 
        self.model.fc = nn.Linear(self.model.fc.in_features, latent_dim)
    
    def _forward(self, x):
      out = self.model(x)
      # normalize embedding to unit vector
      out = torch.nn.functional.normalize(out) 
      return out

    def forward(self, anchor, positive, negative, latent_dim):
        output1 = self._forward(anchor)
        output2 = self._forward(positive)
        output3 = self._forward(negative)
        
        return output1, output2, output3

### Dataloaders

Загрузчики данных не отличаются от загрузчиков для обычной сети. 
Единственное отличие это две аугментации, которые добавили к данным:

*   Случайный поворот по вертикали (RandomHorizontalFlip)
*   Размытие (GaussianBlur)


In [ ]:
from torchvision import transforms as img_transf

# Apply augmentations on train data
img_trans_train = img_transf.Compose(
    [
        img_transf.Resize((105, 105)),
        img_transf.RandomHorizontalFlip(),
        img_transf.GaussianBlur(3, sigma=(0.1, 2.0)),
        img_transf.ToTensor(),
    ]
)

img_trans_test = img_transf.Compose(
    [img_transf.Resize((105, 105)), img_transf.ToTensor()]
)

train_dataset = SiameseNetworkDataset("faces/training/", transform=img_trans_train)
val_dataset = SiameseNetworkDataset("faces/testing/", transform=img_trans_test)

batch_size = 300
train_loader = DataLoader(train_dataset, num_workers=2, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, num_workers=2, batch_size=1, shuffle=False) 

### Обучение

Отличие от  сетей для классификации в том, что у модели 3 выхода и все их надо передать в loss. При этом нет меток в явном виде.
Определить, какой embedding относится к позитивному образцу, а какой к негативному, можно только порядком их следования.

In [ ]:
def train(num_epochs, model, criterion, optimizer, train_loader, latent_dim):
    loss_history = []
    l = []
    model.train()
    for epoch in range(0, num_epochs):
        
        for i, batch in enumerate(train_loader, 0):
            anc, pos, neg = batch
            output_anc, output_pos, output_neg = model(anc.to(device), pos.to(device), neg.to(device), latent_dim)
            loss = criterion(output_anc, output_pos, output_neg)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            l.append(loss.item())
        last_epoch_loss =  torch.tensor(l[-len(train_loader):-1]).mean()
        print("Epoch {} with {:.4f} loss".format(epoch, last_epoch_loss))
        
    return l, last_epoch_loss

При использовании GPU, обучение на 5-ти эпохах займет около 15 сек:

In [ ]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
latent_dim = 32
model = SiameseNet(latent_dim).to(device)
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) 
epochs = 6
l, _ = train(epochs, model, criterion, optimizer, train_loader, latent_dim)

Выведем график loss

In [ ]:
plt.plot([i for i in range(len(l))], l)
plt.ylabel('loss')
plt.xlabel('num of epochs')
x = range(0, len(l), len(l) // epochs)
labels = range(0, epochs)
plt.xticks(x, labels)
plt.grid()
plt.show()

Видно, что он график лосс уже вышел на плато, значит можно проверять результат.

### Проверка

Для начала выведем тройки изображений из проверочного датасета и посмотрим на расстояния для позитивных и негативных пар. Если модель обучилась, расстояния для позитивных пар будут меньше. Другими словами: расстояние между похожими лицами будет маленьким, а между разными будет большим.

P.S. По умолчанию TripletLoss минимизирует Евклидово расстояние.

In [ ]:
# Helper method for visualization
def show(img, text=None):
    img_np = img.numpy()
    plt.axis("off")
    plt.text(75, 120, text, fontweight="bold")
    plt.imshow(np.transpose(img_np, (1, 2, 0)))
    plt.show()

def plot_imgs(model, test_loader, latent_dim):
    distances_pos = []
    distances_neg = []
    # print(f'latent_dim: {latent_dim}')  # if you want to check latent_dim
    model.eval()
    with torch.inference_mode():
      for i, batch in enumerate(test_loader, 0):
        anc, pos, neg = batch
        output_anc, output_pos, output_neg = model(anc.to(device), pos.to(device), neg.to(device), latent_dim)
        # compute euc. distance
        distance_pos = F.pairwise_distance(output_anc, output_pos).item() 
        distance_neg = F.pairwise_distance(output_anc, output_neg).item() 

        distances_pos.append(distance_pos)
        distances_neg.append(distance_neg)

        if not i % 5:
                concatenated = torch.cat((anc, pos, neg))
                result = "OK" if distance_neg - distance_pos > 0 else "BAD"
                show(    
                    torchvision.utils.make_grid(concatenated),
                    f"Positive / negative euclidean distances: {distance_pos:.3f} / {distance_neg:.3f} - {result}",
                )

    return distances_pos, distances_neg


distances_pos, distances_neg = plot_imgs(model, val_loader, latent_dim)


Если модель обучилась, расстояния для позитивных пар будут меньше чем для негативных.

Но такая оценка субъективна, давайте посмотрим на распределение расстояний по категориям:

In [ ]:
!pip install -q pandas==1.1.5

In [ ]:
import seaborn as sns

distances = {"The same person": distances_pos, "Another person": distances_neg}

ax = sns.displot(distances, kde=True, stat="density")
ax.set(xlabel="Pairwise distance")
plt.show()

Видно, что для фото одного и того же человека, в большинстве случаев расстояние лежит в интервале от 0 до 1.25. 

А вот для фото разных людей от 0.5 до 1.75. 

Если бы мы проектировали систему распознавания лиц, нужно было бы выбрать порог чтобы сравнивать с ним расстояние и принимать решение о том верифицировать фото как подлинное или нет.

Соответственно, для нашего игрушечного датасета такой порог следует выбирать ~= 0.75 при условии, что ошибки первого и второго рода для нас равнозначны.

## Few-shot learning in GPT

Языковые модели, такие как GPT-3, тренируются решать одну задачу: предсказывать следующее слово (токен) для текста. На первый взгляд, применений им немного - можно генерировать псведо-осмысленный текст для увеличения объема курсовой работы.

Языковые модели содержат в своих весах знания об обучающем датасете. Чем больше весов, тем больше знаний. Огромные модели, такие как GPT-3 (175 млрд. весов), знают очень многое. Рассмотрим, как использовать эти знания для решения реальных задач.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/gpt_train_inference.png" width="600">

Обученную языковую модель можно **обусловить на решение конкретной задачи** путем встраивания этой задачи в текстовый запрос на входе модели. Для этого необходимо сформулировать запрос таким образом, чтобы ожидаемый ответ был органичным продолжением запроса в среднестатистической книге или статье на соответствующую тему из обучающего датасета.

Например, если нужен рецепт яблочного пирога, то запрос "модель, прошу, выдай мне рецепт яблочного пирога" вряд ли поможет. Надо сформировать запрос так, чтобы он был похож на начало абзаца поваренной книги, например: "Рецепт яблочного пирога. Нам потребуются".

При таком использовании модели не происходит ее обучения или дообучения для определенной задачи - **веса не обновляются**, модель используется "как есть" и распознает задачу как часть входного запроса.

### Zero-shot learning

Zero-shot learning - это формулировка задачи без примеров ее решения.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/gpt_zero_shot.png" width="400">

Попробуем перефразировать первое предложение из оригинальной статьи о GPT-3 [Language Models are Few-Shot Learners (Brown et al., 2020)](https://arxiv.org/abs/2005.14165). Для этого используем простую конструкцию запроса:

> Original: ...
> 
> Paraphrase:

Оригинальная GPT-3 недоступна для свободного использования. Поэтому будем использовать [open-source версию GPT-J](https://6b.eleuther.ai/) от сообщества ElutherAI.

Далее жирным шрифтом выделен запрос, а курсивом - сгенерированный текст.

> **Original: Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task.**
> 
> **Paraphrase:** *abstract is paraphrased content, that is, the work is focused on explaining the rationale and the implications of the research in the simplest and most direct way possible.*
> 
> *Traditionally, there are three forms of data available for training machine learning systems: labeled data, unlabeled data and weak supervision. We survey the recent literature on data, particularly weak supervision, as a potential source of supervision for named entity recognition (NER). Our survey focuses on the challenges of weak supervision and presents a taxonomy of types of weak supervision and their relationships. We also present an evaluation of the different types of weak supervision and show that all*

Модель сгенерировала связный текст на тему, соответствующую запросу. Но ответ нельзя назвать правильным - мы хотели получить перефразированное предложение, а получили свободное рассуждение.

Это случилось потому, что мы не дали модели информацию о желаемом формате ответа. При такой нечеткой формулировке вопроса человек мог бы ошибиться так же, как и модель.

Zero-shot learning может успешно применяться для решения задач, которые требуют лишь извлечения знаний из модели без смысловой обработки. Например, для получения рецепта яблочного пирога.

### One-shot learning

One-shot learning - это формулировка задачи для модели с единственным примером верного решения аналогичной задачи. Добавим в начало запроса пример удачного перефразирования.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/gpt_one_shot.png" width="400">

>**Original: Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.**
>
>**Paraphrase: A giraffe can eat up to 75 pounds of Acacia leaves and hay daily.**
>
>**Original: Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task.**
>
>**Paraphrase:** *Research recently demonstrated large performance improvements on many NLP tasks and benchmarks by pre-training a deep network from scratch, before fine-tuning on the task of interest.*`
>
>`*Origenation note from authors: Some source-sensitivity was lost in this article, as there were some previous iterations of annotations based on the contributors' original source text.*

Единственного примера хватило модели для того, чтобы сгенерировать удачную перефразировку. Но у модели не хватило сведений для того, чтобы понять, на каком месте следует закончить генерацию.

### Few-shot learning

Few-shot learning - это формулировка задачи для модели с несколькими примерами верного решения аналогичных задач.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/gpt_few_shot.png" width="400">

>**Original: Her life spanned years of incredible change for women as they gained more rights than ever before.**
>
>**Paraphrase: She lived through the exciting era of women's liberation.**
>
>**Original: Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.**
>
>**Paraphrase: A giraffe can eat up to 75 pounds of Acacia leaves and hay daily.**
>
>**Original: Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task.**
>
>**Paraphrase:** *Recent work has demonstrated that training on a large corpus of text followed by fine-tuning on a specific task has led to considerable gains.*
>
>*Original: In her book, The Hands-On Teacher, Koehler provides suggestions for the following instructional strategies to help students gain insight into the material:*
>
>*Paraphrase: In her book, The Hands-On Teacher, Koehler provides suggestions for the following instructional strategies to help students gain insight into the material:*

По двум примерам модель уловила закономерности:
- в тексте есть структура "вопрос-ответ", которую надо продолжить
- ответ должен быть близок по смыслу и по длине к вопросу

Модель сгенерировала хороший ответ на заданный вопрос, а потом сама придумала следующий вопрос.

Модель может работать даже с несложными числовыми рядами.

>**Sequence: 2, 4, 6, 8**
>
>**Continuation: 16, 32, 64, 128**
>
>**Sequence: 3, 9, 27, 81**
>
>**Continuation: 243, 729, 2187, 6561**
>
>**Sequence: 4, 16, 64, 256**
>
>**Continuation:** *4096, 16384, 65536, 262144*
>
>*Sequence: 5, 25, 125, 625*
>
>*Continuation: 78125, 390625, 1571875, 587134375*

### Ограничения языковых моделей

Предобученные мощные языковые модели можно использовать для быстрого решения разнообразных сложных задач. Но полученным ответам нельзя слепо доверять - часто модели используют совершенно не те закономерности, которые ожидает пользователь.

Кроме того, ответы имеют стохастический характер. Поэтому иногда даже при качественных запросах результаты будут неудовлетворительными из-за случайности. Например, при предсказании числового ряда правильный результат был получен не с первого раза.

## Оптимизация гиперпараметров

Часто, когда мы пишем и обучаем сети (будь то с нуля или с помощью transfer learning) мы вынуждены угадывать гиперпараметры (lr, betas и тд). В случае с learning rate нам есть от чего оттолкнуться (маленький lr для transfer learning),но все же, такой подход не кажется оптимальным.

Для оптимизации гиперпараметров существуют готовые решения, которые используют различные методы black-box оптимизации. Разберем одну из наиболее популярных библиотек - [**Optuna**](https://optuna.org/)

In [ ]:
!pip install --quiet optuna

Давайте оптимизируем learning rate и latent_dim. Для того, чтобы код не выполнялся очень долго, укажем небольшой диапазон параметров и небольшое число эпох

In [ ]:
import optuna
from optuna.samplers import RandomSampler

# define function which will optimized
def objective(trial):
    # boundaries for the optimizer's 
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    latent_dim = trial.suggest_int("latent_dim", 8, 64, step=8)


    # create new model(and all parameters) every iteration
    model = SiameseNet(latent_dim).to(device)
    criterion = nn.TripletMarginLoss()
    optimizer = optim.Adam(
        model.parameters(), lr=lr
    )  # learning step regulates by optuna
    

    # To save time, we will take only 3 epochs
    train_loader = DataLoader(train_dataset, num_workers=2, batch_size=batch_size, shuffle=True)
    _, last_epoch_loss = train(3, model, criterion, optimizer, train_loader, latent_dim)
    return last_epoch_loss


# Create "exploration"
study = optuna.create_study(direction="minimize", study_name="Optimizer", sampler=RandomSampler(42))

study.optimize(
    objective, n_trials=10
)  # The more iterations, the higher the chances of catching the most optimal hyperparameters

Как видите, такой упрощенный подбор даже трех параметров модели, занимает много времени.

In [ ]:
# show best params
study.best_params

Давайте посмотрим на историю оптимизации наших параметров

In [ ]:
optuna.visualization.plot_optimization_history(study)

Что ж, проверим, а станет ли реально лучше

In [ ]:
model = SiameseNet(study.best_params["latent_dim"]).to(device)
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(
    model.parameters(), lr=study.best_params["lr"]
)  # take lr, which choosen Optuna

epochs = 6
train_loader = DataLoader(train_dataset, num_workers=2, batch_size=batch_size, shuffle=True)   # take batch_size", which choosen Optuna
l_optim, _ = train(epochs, model, criterion, optimizer, train_loader, study.best_params["latent_dim"])  # take latent_dim, which choosen Optuna

In [ ]:
plt.plot([i for i in range(len(l))], l, label="no optimization")
plt.plot([i for i in range(len(l_optim))], l_optim, label="optimal params")
plt.ylabel('loss')
plt.xlabel('num of epochs')
plt.xticks(x, labels)
plt.grid()
plt.legend()
plt.show()

In [ ]:
distances_pos, distances_neg = plot_imgs(model, val_loader, study.best_params["latent_dim"])

In [ ]:
distances_optim = {"The same person": distances_pos, "Another person": distances_neg}

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(distances, kde=True, stat="density", alpha=0.5, ax=axes[0])
sns.histplot(distances_optim, kde=True, stat="density", alpha=0.5, ax=axes[1])

axes[0].set(title='No optimization')
axes[1].set(title='Otimization with Optuna')
axes[0].set(xlabel="Pairwise distance")
axes[1].set(xlabel="Pairwise distance")

plt.show()

Распределение расстояний стало лучше, хотя, по-прежнему, не идеально.

Это объясняется тем, что 10 итераций, для подбора гиперпараметров, все-таки маловато.

<font size = "6"> Заключение


Мы затронули проблемы, которые возникают при обучении на реальных данных.

Одна из основных проблем - малые датасеты. Для того, чтобы обучить нейронку на небольшом датасете, можно использовать:
- `аугментацию`
- `Tranfer learning`
Однако необходимо помнить, что ни один из этих методов не защитит от ситуации, когда реальные данные будут сильно отличаться от тренировочных.

В случае, когда у нас не только мало данных, но и еще и очень большое (возможно, неизвестное) число классов, можно воспользоваться `One-shot Learning`. В этом случае нейронка обучается не классифицировать изображения, а, наоборот, находить различия между классом и новыми данными. Для этого используются нейронные сети, относящиеся к классу сиамских нейронных сетей.

Так же мы разобрали автоматическую оптимизацию гиперпараметров с помощью Optuna и показали, что это эффективный способ сделать нейросеть лучше малой ценой.

<font size = "6">Литература


<font size = "5"> Обучение на реальных данных

[How to avoid machine learning pitfalls: a guide for academic researchers (Lones, 2021)](https://arxiv.org/abs/2108.02497)

[Understanding data augmentation for classification: when to warp? (Wong et al., 2016)](https://arxiv.org/abs/1609.08764) 

[Learning from class-imbalanced data: Review of methods and applications (Haixiang et al., 2017)](https://www.sciencedirect.com/science/article/abs/pii/S0957417416307175?via%3Dihub)

<font size = "5"> Как решить проблему маленького количества данных?

[Блог-пост о том, как решать проблему малого количества данных.](https://towardsdatascience.com/breaking-the-curse-of-small-datasets-in-machine-learning-part-1-36f28b0c044d)

<font size = "5"> Несбалансированные данные

[Imbalanced Data: How to handle Imbalanced Classification Problems](https://medium.com/@cmukesh8688/how-to-handle-imbalanced-classification-problems-4a96f42ae4c4).

[SMOTE explained for noobs - Synthetic Minority Over-sampling TEchnique line by line](https://rikunert.com/SMOTE_explained)

[Блог пост про 8 тактик борьбы с несбалансированными классами в наборе данных машинного обучения](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)

[Метрики, разработаные для работы с несбалансированными классами.](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/#:~:text=Classification%20Accuracy%20is%20Not%20Enough%3A%20More%20Performance%20Measures%20You%20Can%20Use,-By%20Jason%20Brownlee&text=When%20you%20build%20a%20model,This%20is%20the%20classification%20accuracy)

[Творческий подход](https://www.quora.com/In-classification-how-do-you-handle-an-unbalanced-training-set)

<font size = "5"> Transfer Learning

[Image Classification using Transfer Learning in Pytorch](https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/)

[How To Do Transfer Learning For Computer Vision | PyTorch Tutorial](https://www.youtube.com/watch?v=6nQlxJvcTr0)

[Transfer learning for Computer Vision Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

[Python Pytorch Tutorials # 2 Transfer Learning : Inference with ImageNet Models](https://www.youtube.com/watch?v=Upw4RaERZic)

[PyTorch - The Basics of Transfer Learning with TorchVision and AlexNet](https://www.youtube.com/watch?v=8etkVC93yU4)


<font size = "5">Augmentation

[A survey on Image Data Augmentation for Deep Learning (Shorten and Khoshgoftaar, 2019)](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0197-0)

[Data augmentation for improving deep learning in image classification problem](https://www.researchgate.net/publication/325920702_Data_augmentation_for_improving_deep_learning_in_image_classification_problem)


<font size = "5"> Few-shot learning

[One-Shot Learning with Siamese Networks using Keras](https://towardsdatascience.com/one-shot-learning-with-siamese-networks-using-keras-17f34e75bb3d)

[One-Shot image classification by meta learning](https://medium.com/nerd-for-tech/one-shot-learning-fe1087533585)

[One-Shot Learning (Part 1/2): Definitions and fundamental techniques](https://heartbeat.fritz.ai/one-shot-learning-part-1-2-definitions-and-fundamental-techniques-1df944e5836a)

[One-Shot Learning (Part 2/2): Facial Recognition Using a Siamese Network](https://heartbeat.fritz.ai/one-shot-learning-part-2-2-facial-recognition-using-a-siamese-network-5aee53196255)

[FaceNet: A Unified Embedding for Face Recognition and Clustering (Schroff et al., 2015)](https://arxiv.org/abs/1503.03832)

[One-Shot Learning explained using FaceNet](https://medium.com/intro-to-artificial-intelligence/one-shot-learning-explained-using-facenet-dff5ad52bd38)

[Ищем знакомые лица](https://habr.com/ru/post/317798/)

[Siamese Neural Networks for One-shot Image Recognition (Koch et al., 2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf)

[Dimensionality Reduction by Learning an Invariant Mapping (Hadsell et al., 2005)](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)

[Language Models are Few-Shot Learners (Brown et al., 2020)](https://arxiv.org/abs/2005.14165)

<font size = "5"> Hyperparameter optimization

[Tuning Hyperparameters with Optuna](https://towardsdatascience.com/tuning-hyperparameters-with-optuna-af342facc549)